In [ ]:
from pathlib import Path
import pandas as pd

In [ ]:
# Find repo root even if notebook is inside /notebooks
ROOT = Path.cwd()
if ROOT.name == "notebooks":
    ROOT = ROOT.parent

DATA_DIR = ROOT / "data"
DATA_DIR.mkdir(exist_ok=True)

In [ ]:
wind_data = ROOT / "data/SyslabWind_15min.csv"
pv_data = ROOT / "data/SyslabPV_15min.csv"
weather_data = ROOT / "data/SyslabWeather_15min.csv"
price_data = ROOT / "data/DayAheadPrices_04022026_cleaned.csv"

pv_data_nozeros = ROOT / "data/SyslabPV_15min_nozeros.csv"
wind_data_nozeros = ROOT / "data/SyslabWind_15min_nozeros.csv"

SOLETE_data = ROOT / 'data/SOLETE_1sec_15min.csv'
SOLETE_price_data = ROOT / 'data/Elspotprices_SOLETE_cleaned.csv'

In [ ]:
def combine_data(files:list,filename,save_new_file=True):
    df_combined = pd.read_csv(files[0], 
                     delimiter=',', 
                     decimal='.', 
                     parse_dates=['ts'],  # Your timestamp column name
                     index_col='ts')
    
    for file in files[1:]: 
        df_temp = pd.read_csv(file, 
                     delimiter=',', 
                     decimal='.', 
                     parse_dates=['ts'],  # Your timestamp column name
                     index_col='ts')
        df_combined = df_combined.join(df_temp,how="outer") ## outer = keep ALL timestamps included in ALL the files, inner, left and right still exist! (decide perhaps for inner later on)
        print(f"Added {file}, shape is now: {df_combined.shape}")

    if save_new_file == False: 
        return df_combined

    df_combined.to_csv(DATA_DIR / f'{filename}.csv', index=True, sep =",") ## Save to two folders up and then into data ;) 
    print(f"\nFinal combined shape: {df_combined.shape}")
    print(f"Columns: {list(df_combined.columns)}")



In [ ]:
#the combined data now has generation data averaged across 15 mins but without 0; all generation should be greater than 0

combine_data([wind_data_nozeros,pv_data_nozeros,weather_data,price_data],filename="combined_data_positive_gen",save_new_file=True)

In [ ]:
## combine data SOLETE and pricing

combine_data([SOLETE_data, SOLETE_price_data],filename="combined_SOLETE_data",save_new_file=True)